In [1]:
import torch
from tokenizers.implementations import ByteLevelBPETokenizer, BertWordPieceTokenizer
from tokenizers.processors import BertProcessing

from src.models.model import Transformer


In [2]:
MAX_LEN = 256

In [3]:
dev = 'cuda'

In [4]:
weight_fn = 'weights/Transformer-En2Vi-CPE-WordPiece/best_bleu.pth'
config = torch.load(weight_fn, map_location=dev)

In [5]:
data_cfg = config['config']['dataset']['train']['config']
token_type = data_cfg['token_type']
if token_type == 'bpe':
    vi_tokenizer = ByteLevelBPETokenizer(
        "vocab/vietnamese/vietnamese-vocab.json",
        "vocab/vietnamese/vietnamese-merges.txt",
    )

    vi_tokenizer._tokenizer.post_processor = BertProcessing(
        ("</s>", vi_tokenizer.token_to_id("</s>")),
        ("<s>", vi_tokenizer.token_to_id("<s>")),
    )
    vi_tokenizer.enable_truncation(max_length=MAX_LEN)

    en_tokenizer = ByteLevelBPETokenizer(
        "vocab/english/english-vocab.json",
        "vocab/english/english-merges.txt",
    )

    en_tokenizer._tokenizer.post_processor = BertProcessing(
        ("</s>", en_tokenizer.token_to_id("</s>")),
        ("<s>", en_tokenizer.token_to_id("<s>")),
    )
    en_tokenizer.enable_truncation(max_length=MAX_LEN)
elif token_type == 'wordpiece':
    vi_tokenizer = BertWordPieceTokenizer(
        data_cfg['trg_vocab'],
        lowercase=False,
        handle_chinese_chars=True,
        strip_accents=False,
        cls_token='<s>',
        pad_token='<pad>',
        sep_token='</s>',
        unk_token='<unk>',
        mask_token='<mask>',
    )
    vi_tokenizer.enable_truncation(max_length=MAX_LEN)

    en_tokenizer = BertWordPieceTokenizer(
        data_cfg['src_vocab'],
        lowercase=False,
        handle_chinese_chars=True,
        strip_accents=False,
        cls_token='<s>',
        pad_token='<pad>',
        sep_token='</s>',
        unk_token='<unk>',
        mask_token='<mask>',
    )
    en_tokenizer.enable_truncation(max_length=MAX_LEN)


TRG_EOS_TOKEN = '</s>'
TRG_EOS_ID = vi_tokenizer.token_to_id(TRG_EOS_TOKEN)

In [6]:
model = Transformer(
    n_src_vocab=en_tokenizer.get_vocab_size(),
    n_trg_vocab=vi_tokenizer.get_vocab_size(),
    src_pad_idx=en_tokenizer.token_to_id('<pad>'),
    trg_pad_idx=vi_tokenizer.token_to_id('<pad>'),
    **config['config']['model']
).to(dev)
model.load_state_dict(config['model_state_dict'])
model.eval()
print()

In [7]:
@torch.no_grad()
def translate(model, src, dev, en_tokenizer, vi_tokenizer):
    src = torch.tensor(en_tokenizer.encode(src).ids)
    src = src.long().unsqueeze(0).to(dev)
    pred = model.predict(src, max_length=MAX_LEN, eos_id=TRG_EOS_ID)
    pred_ = pred.cpu().numpy()
    pred_ = vi_tokenizer.decode_batch(pred_)[0]
    return pred_

# Random string

In [8]:
src = 'Today is beautiful'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Hôm nay là một nơi tuyệt vời'

In [9]:
src = 'I want to be a doctor in the future'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Tôi muốn trở thành một bác sĩ trong tương lai'

In [10]:
src = 'All I want for Christmas is you'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Tất cả những gì tôi muốn cho các bạn là của tôi.'

In [11]:
src = 'I want you by my side always'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Tôi luôn muốn bạn thấy mặt tôi.'

In [12]:
src = 'My love is for you and you only'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Tôi dành cho các bạn và chỉ có các bạn mà thôi.'

# Order importance

In [13]:
src = 'I want to be a doctor'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Tôi muốn trở thành một bác sĩ.'

In [14]:
src = 'I a doctor want to be'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Tôi muốn một bác sĩ cần được đào tạo.'

In [15]:
src = 'want to be I a doctor'
translate(model, src, dev, en_tokenizer, vi_tokenizer)

'Tôi muốn trở thành bác sĩ tâm thần.'